In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.preprocessing import LabelBinarizer
import json

In [ ]:
def load_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    return img_array

def parse_annotation(annotation_path):
    with open(annotation_path) as f:
        annotations = json.load(f)
    return annotations

def prepare_dataset(images_folder, annotations_file, target_size):
    annotations = parse_annotation(annotations_file)
    images = []
    boxes = []
    labels = []
    
    for annotation in annotations['annotations']:
        image_id = annotation['image_id']
        image_info = next(img for img in annotations['images'] if img['id'] == image_id)
        image_path = os.path.join(images_folder, image_info['file_name'])
        img = load_image(image_path, target_size)
        bbox = annotation['bbox']
        label = annotation['category_id']
        
        images.append(img)
        boxes.append(bbox)
        labels.append(label)
    
    images = np.array(images) / 255.0
    return images, boxes, labels

images_folder = 'archive/car/train'
annotations_file = 'path/to/annotations.json'
target_size = (512, 512)
images, boxes, labels = prepare_dataset(images_folder, annotations_file, target_size)
